# Real-Time Network Emulation Visualization

This notebook provides real-time visualization of the running SiNE emulation.

## Prerequisites

1. **Channel server must be running**: `uv run sine channel-server`
2. **Emulation must be deployed**: `sudo $(which uv) run sine deploy <topology.yaml>`

## How It Works

- Polls the channel server's `/api/visualization/state` endpoint
- Displays cached path data from previous channel computations
- Shows wireless channel metrics (delay spread, K-factor, coherence bandwidth)
- Updates every 1 second (configurable)

**Important**: No computation is performed in this notebook - all data is pre-computed and cached by the channel server during emulation deployment/updates.

In [ ]:
# Cell 1: Setup and Configuration
import asyncio
import httpx
from IPython.display import clear_output, display
from typing import Any

# Configuration
CHANNEL_API = "http://localhost:8000"
UPDATE_INTERVAL_SEC = 1.0  # Poll interval
MAX_RENDER_PATHS = 5       # Limit paths per link for performance

In [ ]:
# Cell 2: Helper Functions
async def fetch_visualization_state() -> dict[str, Any]:
    """
    Fetch complete visualization state from channel server.

    Returns scene geometry, device positions, and CACHED paths.
    No computation required - instant response.
    """
    async with httpx.AsyncClient(timeout=5.0) as client:
        response = await client.get(f"{CHANNEL_API}/api/visualization/state")
        response.raise_for_status()
        return response.json()

In [ ]:
# Cell 3: Scene Preview with Paths (Option 2: Re-compute paths in notebook)
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, PathSolver
import numpy as np
from typing import Optional
from pathlib import Path

def render_scene_with_paths(viz_state: dict[str, Any], clip_at: Optional[float] = None) -> None:
    """
    Render 3D scene with devices and propagation paths using Sionna preview.
    
    Implementation: Option 2 (Re-compute paths for visualization)
    - Uses cached device positions from channel server
    - Re-runs PathSolver in notebook to get Paths object
    - Small overhead acceptable for snapshot/infrequent visualization
    
    Args:
        viz_state: Visualization state from channel server
        clip_at: Optional z-coordinate to clip scene (useful for indoor scenes)
    """
    scene_file = viz_state.get('scene_file')
    if not scene_file:
        print("No scene file available - text visualization only")
        return
    
    try:
        # Resolve scene path - handle both absolute and relative paths
        scene_path = Path(scene_file)
        if not scene_path.is_absolute():
            notebook_dir = Path.cwd()
            if notebook_dir.name == 'scenes':
                project_root = notebook_dir.parent
            else:
                project_root = notebook_dir
            scene_path = project_root / scene_file
        
        if not scene_path.exists():
            print(f"Scene file not found: {scene_path}")
            print(f"Current directory: {Path.cwd()}")
            print("Continuing with text-only visualization...")
            return
        
        # Load scene (merge_shapes=False to keep surfaces separate)
        scene = load_scene(str(scene_path), merge_shapes=False)
        
        # Configure minimal antenna arrays for visualization
        scene.tx_array = PlanarArray(
            num_rows=1, num_cols=1,
            vertical_spacing=0.5, horizontal_spacing=0.5,
            pattern="iso", polarization="V"
        )
        scene.rx_array = PlanarArray(
            num_rows=1, num_cols=1,
            vertical_spacing=0.5, horizontal_spacing=0.5,
            pattern="iso", polarization="V"
        )
        
        # Set frequency (important for material properties)
        scene.frequency = 5.18e9  # 5.18 GHz (WiFi 5)
        
        # Add devices from cached positions
        tx_added = set()
        rx_added = set()
        
        for link in viz_state["paths"]:
            tx_name = link["tx_name"]
            rx_name = link["rx_name"]
            tx_pos = link["tx_position"]
            rx_pos = link["rx_position"]
            
            # Add TX if not already added
            if tx_name not in tx_added:
                scene.add(Transmitter(name=tx_name, position=tx_pos))
                tx_added.add(tx_name)
            
            # Add RX if not already added
            if rx_name not in rx_added:
                scene.add(Receiver(name=rx_name, position=rx_pos))
                rx_added.add(rx_name)
        
        print(f"\n{'='*70}")
        print("3D SCENE PREVIEW WITH PROPAGATION PATHS")
        print(f"{'='*70}")
        print(f"Scene: {scene_path}")
        print(f"Devices: {len(tx_added)} TX, {len(rx_added)} RX")
        print(f"Links: {len(viz_state['paths'])}")
        print("\nComputing propagation paths for visualization...")
        print("(Note: Paths are re-computed from cached positions)")
        
        # Re-compute paths using PathSolver
        # This is redundant (already computed for netem), but necessary
        # to get a Sionna Paths object for scene.preview()
        solver = PathSolver()
        paths = solver(scene)
        
        print("Paths computed successfully!")
        print(f"{'='*70}\n")
        
        # Preview scene with paths and clipping
        if clip_at is not None:
            scene.preview(paths=paths, clip_at=clip_at)
        else:
            scene.preview(paths=paths)
            
    except Exception as e:
        print(f"Failed to render scene: {e}")
        import traceback
        traceback.print_exc()
        print(f"Scene path attempted: {scene_path if 'scene_path' in locals() else scene_file}")
        print(f"Current directory: {Path.cwd()}")

In [ ]:
# Cell 4: Text Display with Wireless Metrics
def display_text_summary(viz_state: dict[str, Any]) -> None:
    """
    Display text summary of visualization state with wireless channel analysis.
    
    Args:
        viz_state: Visualization state from channel server
    """
    print(f"=== Visualization State (Cache: {viz_state['cache_size']} links) ===")
    print(f"Scene: {viz_state.get('scene_file', 'N/A')}\n")

    # Display device positions
    print("Devices:")
    for device in viz_state["devices"]:
        pos = device["position"]
        print(f"  {device['name']}: ({pos['x']:.1f}, {pos['y']:.1f}, {pos['z']:.1f})")

    # Display detailed channel information
    print(f"\n{'='*70}")
    print("WIRELESS CHANNEL ANALYSIS")
    print(f"{'='*70}")

    for link_data in viz_state["paths"]:
        print(f"\nLink: {link_data['tx_name']} → {link_data['rx_name']}")
        print(f"{'-'*70}")

        # Basic link info
        print(f"Distance: {link_data['distance_m']:.1f} m")
        print(f"Paths: {link_data['num_paths_shown']}/{link_data['num_paths_total']} "
              f"({link_data.get('power_coverage_percent', 100):.1f}% power)")

        # Delay spread analysis (ISI characterization)
        rms_ds_ns = link_data.get('rms_delay_spread_ns', 0)
        bc_mhz = link_data.get('coherence_bandwidth_hz', 0) / 1e6

        print(f"\nDelay Characteristics:")
        print(f"  RMS Delay Spread (τ_rms): {rms_ds_ns:.2f} ns")
        print(f"  Coherence Bandwidth (Bc): {bc_mhz:.1f} MHz")

        # Frequency selectivity assessment
        # Assume 80 MHz signal BW (adjust based on your config)
        signal_bw_mhz = 80  # TODO: Get from link config
        if bc_mhz > signal_bw_mhz:
            print(f"  ✓ Frequency-flat channel (Bc > BW)")
        else:
            print(f"  ⚠ Frequency-selective channel (Bc ≈ BW)")
            print(f"    ISI may be significant - OFDM recommended")

        # LOS/NLOS classification via Rician K-factor
        k_factor = link_data.get('k_factor_db')
        dominant_type = link_data.get('dominant_path_type', 'unknown')

        print(f"\nChannel Classification:")
        if k_factor is not None:
            print(f"  Rician K-factor: {k_factor:.1f} dB")
            if k_factor > 10:
                print(f"  → Strong LOS component (K > 10 dB)")
            elif k_factor > 0:
                print(f"  → Moderate LOS with multipath (0 < K < 10 dB)")
            else:
                print(f"  → NLOS dominant (K < 0 dB)")
        else:
            print(f"  Channel Type: NLOS (no direct path)")
            print(f"  Dominant: {dominant_type}")

        # Individual path details
        print(f"\nPropagation Paths (strongest {link_data['num_paths_shown']}):")
        for i, path in enumerate(link_data['paths'], 1):
            los_marker = " [LOS]" if path['is_los'] else ""
            interactions = ", ".join(path['interaction_types']) if path['interaction_types'] else "direct"
            doppler = f", Doppler: {path.get('doppler_hz', 0):.1f} Hz" if path.get('doppler_hz') is not None else ""

            print(f"  Path {i}: {path['delay_ns']:.2f} ns, {path['power_db']:.1f} dB{los_marker}")
            print(f"          Interactions: {interactions}{doppler}")

In [ ]:
# Cell 5: One-Time Snapshot with 3D Preview
async def render_snapshot(show_3d: bool = True, clip_at: Optional[float] = 2.0) -> None:
    """
    Render a single snapshot of current visualization state.
    
    Args:
        show_3d: Whether to show 3D scene preview (default: True)
        clip_at: Z-coordinate to clip scene at (default: 2.0m for indoor scenes)
    
    Use this instead of the continuous loop if you just want
    to see the current state once.
    """
    # Fetch current state
    viz_state = await fetch_visualization_state()

    # Display 3D scene preview if requested
    if show_3d:
        render_scene_with_paths(viz_state, clip_at=clip_at)
    
    # Display text summary
    display_text_summary(viz_state)

# Run snapshot (with 3D preview clipped at 2m height)
await render_snapshot(show_3d=True, clip_at=2.0)

In [ ]:
# Cell 6: Simple Scene Test with Paths (Diagnose preview issues)
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, PathSolver
from pathlib import Path

# Simple test: load scene, add devices, compute paths, and preview
scene_path = Path("/home/joshua/Documents/SiNE/scenes/two_rooms.xml")

print(f"Loading scene from: {scene_path}")
print(f"File exists: {scene_path.exists()}")

scene = load_scene(str(scene_path), merge_shapes=False)
print(f"Scene loaded successfully with {len(scene.objects)} objects")

# Configure arrays
scene.tx_array = PlanarArray(num_rows=1, num_cols=1, pattern="iso", polarization="V")
scene.rx_array = PlanarArray(num_rows=1, num_cols=1, pattern="iso", polarization="V")
scene.frequency = 5.18e9  # 5.18 GHz (WiFi 5)

# Add TX in Room 1 and RX in Room 2 (matching two_rooms example positions)
scene.add(Transmitter(name="tx1", position=[10.0, 10.0, 1.0]))
scene.add(Receiver(name="rx1", position=[30.0, 10.0, 1.0]))

print("\nComputing propagation paths...")
# Compute paths using PathSolver (required to visualize paths)
solver = PathSolver()
paths = solver(scene)

print(f"Paths computed successfully!")
print(f"\nCalling scene.preview(paths=paths, clip_at=2.0)...")
print("Note: If you don't see a 3D viewer, your Jupyter environment may not support interactive preview.")
print("Try running this notebook in standard Jupyter Notebook (not VS Code) if the preview doesn't appear.")
print("\nExpected: You should see propagation paths (lines) between the green TX and blue RX devices.")

# This should trigger the preview WITH PATHS visible
scene.preview(paths=paths, clip_at=2.0)

In [ ]:
# Cell 7: Create Animation Movie from Scene Rendering
import asyncio
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import numpy as np
from pathlib import Path
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, PathSolver, Camera
from typing import Any, List
import io

async def create_channel_movie(
    t_monitor: float = 30.0,
    delta_t: float = 1.0,
    clip_at: float = 2.0,
    resolution: tuple[int, int] = (800, 600),
    num_samples: int = 32,
    camera_position: tuple[float, float, float] = (40.0, 40.0, 45.0),
    camera_look_at: tuple[float, float, float] = (20.0, 20.0, 0.0),
    fov: float = 70.0
) -> HTML:
    """
    Create an animation movie of channel state over the past t_monitor seconds.
    
    This function:
    1. Polls the channel server at delta_t intervals for t_monitor seconds
    2. Renders the 3D scene with devices and paths at each time step using scene.render()
    3. Saves rendered images to memory
    4. Creates a playable matplotlib animation
    
    Args:
        t_monitor: Total monitoring duration in seconds (default: 30s)
        delta_t: Time interval between frames in seconds (default: 1s)
        clip_at: Z-coordinate to clip scene at (default: 2.0m)
        resolution: Image resolution as (width, height) (default: 800x600)
        num_samples: Number of rendering samples for quality (default: 32, lower=faster)
        camera_position: Camera position (x, y, z) in meters (default: 20, 20, 15)
        camera_look_at: Point to look at (x, y, z) in meters (default: 20, 20, 0)
        fov: Field of view in degrees (default: 45)
    
    Returns:
        HTML video player widget for Jupyter
    
    Example:
        # Create 30-second movie with 1-second intervals (30 frames)
        movie = await create_channel_movie(t_monitor=30.0, delta_t=1.0)
        
        # Create 60-second movie with 2-second intervals (30 frames)
        movie = await create_channel_movie(t_monitor=60.0, delta_t=2.0)
        
        # Fast rendering (lower quality, faster capture)
        movie = await create_channel_movie(t_monitor=30.0, delta_t=1.0, num_samples=16)
    """
    
    num_frames = int(t_monitor / delta_t)
    frames = []
    timestamps = []
    
    print(f"{'='*70}")
    print(f"CREATING CHANNEL ANIMATION MOVIE")
    print(f"{'='*70}")
    print(f"Monitoring duration: {t_monitor}s")
    print(f"Frame interval: {delta_t}s")
    print(f"Total frames: {num_frames}")
    print(f"Resolution: {resolution[0]}x{resolution[1]}")
    print(f"Render quality: {num_samples} samples (lower=faster)")
    print(f"{'='*70}\n")
    
    # Pre-fetch scene info
    print("Fetching initial scene configuration...")
    initial_state = await fetch_visualization_state()
    scene_file = initial_state.get('scene_file')
    
    if not scene_file:
        print("ERROR: No scene file available for rendering")
        return None
    
    # Resolve scene path
    scene_path = Path(scene_file)
    if not scene_path.is_absolute():
        notebook_dir = Path.cwd()
        if notebook_dir.name == 'scenes':
            project_root = notebook_dir.parent
        else:
            project_root = notebook_dir
        scene_path = project_root / scene_file
    
    if not scene_path.exists():
        print(f"ERROR: Scene file not found: {scene_path}")
        return None
    
    print(f"Scene file: {scene_path}")
    
    # Create camera object
    camera = Camera(position=camera_position, look_at=camera_look_at)
    
    print(f"\nStarting frame capture (this will take ~{t_monitor}s + rendering time)...")
    print(f"Estimated total time: ~{t_monitor + num_frames * num_samples * 0.05:.1f}s")
    print("Progress: ", end='', flush=True)
    
    # Capture frames over monitoring period
    for i in range(num_frames):
        # Fetch current visualization state
        try:
            viz_state = await fetch_visualization_state()
        except Exception as e:
            print(f"\nWarning: Failed to fetch state at frame {i}: {e}")
            continue
        
        # Render scene to image
        try:
            frame_img = render_scene_frame(
                viz_state=viz_state,
                scene_path=scene_path,
                camera=camera,
                clip_at=clip_at,
                resolution=resolution,
                num_samples=num_samples,
                fov=fov
            )
            
            frames.append(frame_img)
            timestamps.append(i * delta_t)
            print('█', end='', flush=True)
            
        except Exception as e:
            print(f"\nWarning: Failed to render frame {i}: {e}")
            import traceback
            traceback.print_exc()
            continue
        
        # Wait for next frame (except on last iteration)
        if i < num_frames - 1:
            await asyncio.sleep(delta_t)
    
    print(f"\n\nCapture complete! Captured {len(frames)} frames")
    
    if len(frames) == 0:
        print("ERROR: No frames captured")
        return None
    
    # Create matplotlib animation
    print("\nCreating animation...")
    fig, ax = plt.subplots(figsize=(10, 7.5))
    ax.axis('off')
    
    # Initialize with first frame
    im = ax.imshow(frames[0])
    time_text = ax.text(0.02, 0.98, '', transform=ax.transAxes,
                        fontsize=16, color='white', verticalalignment='top',
                        bbox=dict(boxstyle='round', facecolor='black', alpha=0.8))
    
    def update_frame(frame_idx):
        """Update function for animation"""
        im.set_array(frames[frame_idx])
        time_text.set_text(f'Time: {timestamps[frame_idx]:.1f}s / {t_monitor:.1f}s')
        return [im, time_text]
    
    # Create animation
    anim = FuncAnimation(
        fig, 
        update_frame, 
        frames=len(frames),
        interval=delta_t * 1000,  # Convert to milliseconds
        blit=True,
        repeat=True
    )
    
    plt.close(fig)  # Don't display static figure
    
    print("Animation created successfully!")
    print(f"\nPlayback info:")
    print(f"  - Frames: {len(frames)}")
    print(f"  - Duration: {t_monitor}s")
    print(f"  - Frame rate: {1/delta_t:.1f} fps")
    print(f"  - Loop: Enabled")
    print(f"\nDisplaying movie player...")
    print(f"{'='*70}\n")
    
    # Return HTML5 video widget
    return HTML(anim.to_html5_video())


def render_scene_frame(
    viz_state: dict[str, Any],
    scene_path: Path,
    camera: Camera,
    clip_at: float,
    resolution: tuple[int, int],
    num_samples: int,
    fov: float
) -> np.ndarray:
    """
    Render a single frame using Sionna's scene.render() method.
    
    Args:
        viz_state: Visualization state from channel server
        scene_path: Path to scene XML file
        camera: Camera object
        clip_at: Clipping plane z-coordinate
        resolution: (width, height) tuple
        num_samples: Number of rendering samples
        fov: Field of view in degrees
    
    Returns:
        RGB image as numpy array (H, W, 3) with values in [0, 255]
    """
    # Load scene
    scene = load_scene(str(scene_path), merge_shapes=False)
    
    # Configure antenna arrays
    scene.tx_array = PlanarArray(
        num_rows=1, num_cols=1,
        vertical_spacing=0.5, horizontal_spacing=0.5,
        pattern="iso", polarization="V"
    )
    scene.rx_array = PlanarArray(
        num_rows=1, num_cols=1,
        vertical_spacing=0.5, horizontal_spacing=0.5,
        pattern="iso", polarization="V"
    )
    scene.frequency = 5.18e9
    
    # Add devices from visualization state
    tx_added = set()
    rx_added = set()
    
    for link in viz_state["paths"]:
        tx_name = link["tx_name"]
        rx_name = link["rx_name"]
        tx_pos = link["tx_position"]
        rx_pos = link["rx_position"]
        
        if tx_name not in tx_added:
            scene.add(Transmitter(name=tx_name, position=tx_pos))
            tx_added.add(tx_name)
        
        if rx_name not in rx_added:
            scene.add(Receiver(name=rx_name, position=rx_pos))
            rx_added.add(rx_name)
    
    # Compute paths
    solver = PathSolver()
    paths = solver(scene)
    
    # Render using scene.render() - pass camera object directly
    # scene.render() accepts Camera object or string name
    fig = scene.render(
        camera=camera,
        paths=paths,
        clip_at=clip_at,
        resolution=resolution,
        num_samples=num_samples,
        fov=fov,
        show_devices=True,
        return_bitmap=False  # Returns matplotlib Figure
    )
    
    # Extract image from matplotlib figure
    # Convert figure canvas to numpy array (modern matplotlib API)
    fig.canvas.draw()
    
    # Get the RGBA buffer from the figure (modern API)
    width, height = fig.canvas.get_width_height()
    buf = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    buf = buf.reshape((height, width, 4))
    
    # Convert RGBA to RGB (drop alpha channel)
    buf = buf[:, :, :3]
    
    plt.close(fig)  # Clean up figure
    
    return buf


# Example usage:
# Create a 30-second movie with 1-second intervals
# movie = await create_channel_movie(t_monitor=30.0, delta_t=1.0)
# display(movie)

# Or create a 60-second movie with 2-second intervals (faster rendering)
# movie = await create_channel_movie(t_monitor=60.0, delta_t=2.0, num_samples=16)
# display(movie)

In [ ]:
# Cell 8: Run Movie Creation
# Create a 30-second movie with 1-second intervals
# Adjust parameters as needed for your use case

movie = await create_channel_movie(
    t_monitor=30.0,      # Monitor for 30 seconds
    delta_t=1.0,         # Capture frame every 1 second
    clip_at=2.0,         # Clip scene at z=2.0m (for indoor scenes)
    num_samples=16,      # Lower samples = faster rendering (16 is good for preview)
    resolution=(800, 600)  # Image resolution
)

# Display the movie player
display(movie)